In [0]:
from os import listdir
from os.path import isfile, join
from PIL import Image

#connect to drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
#go to data path
cd 'drive/My Drive/Colab Notebooks'


/content/drive/My Drive/Colab Notebooks


In [0]:
#!unzip 'data.zip'   used once to uncompress data

In [0]:
#load Data
def load_data(train_path):
    X=np.array([])
    y=[]
    
    for label in listdir(train_path):
        for i in listdir(train_path+label+'/'):
          image_path = train_path+label+'/'+i
          img = cv2.imread(image_path)
          img=cv2.resize(img,(256, 256))
          img=  np.array(img)
          img=img.reshape(1,3,img.shape[0],img.shape[1])
          #print('image shape:',img.shape)
          X = np.vstack([X, img]) if X.size else img
          y.append(label)
          #print(img, label)
    #print("X",X.shape,"y",len(y))
         
    #categorize the labels
    labeincoder_y_1 = LabelEncoder()
    y[: ] = labeincoder_y_1.fit_transform(y[: ])
    X = np.asarray(X)
    #print("Yenconded",y)
    #print("Y",y,len(y))
    #print("X",X,X.shape)
    # split the data into a training set and a test set
    # stratify returns training and test subsets that have the same proportions of class labels as the input dataset.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,stratify=y)  
    X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size = 0.25,stratify=y_train)
    
    #to give weights to balance the classes labels
    class_weights = compute_class_weight('balanced', np.unique(y_train),y_train)
    
    #print("train test",X_train,len(X_train), X_test,len(X_test), y_train, y_test)
    return X,y,X_train, X_test, y_train, y_test,X_dev, y_dev,labeincoder_y_1

In [0]:
def DL_model (data_train, data_test, label_train, label_test,data_valed, label_valed,path_test):
  name=[]
  result=[]
  K.set_image_dim_ordering('th')


  model = Sequential()

  model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(3, 256, 256)))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(9))
  model.add(Activation('softmax'))

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])





  model.fit(data_train, np.asarray(label_train),
            batch_size=32,
            validation_data=(data_valed, np.asarray(label_valed)),
            epochs=100)


  #evaluate model
  loss, acc = model.evaluate(data_test, np.asarray(label_test))
  print("loss:",loss,"accuracy:", acc)
  
  # serialize model to JSON
  model_json = model.to_json()
  with open("model.json", "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights("model.h5")
  print("Saved model to disk")
 
  #classify unlabeled Data
  
  X2 = []
  ids = []
  for i in listdir(path_test+'/'):
      img = cv2.imread(path_test+'/'+i)
      img=cv2.resize(img,(256, 256))
      img=  np.array(img)
      img=img.reshape(3,img.shape[0],img.shape[1])
      X2.append(img)
      ids.append(i)
  X2 = np.asarray(X2)
#       X2 = np.vstack([X2, img]) if X2.size else img
  preds=model.predict_classes(X2, batch_size=64, verbose=0)
  result=label_encoder.inverse_transform(preds)  
#       print(i,result)
#       name.append(i)
#       result.append(result)
  np.savetxt('results.csv', [p for p in zip(ids, result)], delimiter=',', fmt='%s')
      
      


In [0]:
#importing libraries
import cv2
import csv
import numpy as np
from numpy import array
from os import listdir
from os.path import isfile, join
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.utils.class_weight import compute_class_weight
import keras
import keras.utils
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from sklearn.metrics import accuracy_score
from keras.layers.normalization import BatchNormalization

path_train ='train-data/'
path_test  ='test-data/'
data_to_train,label,data_train, data_test, label_train, label_test,data_valed, label_valed,label_encoder=load_data(path_train)
print("train",data_train.shape,"test",data_test.shape,"label",label)

DL_model (data_train, data_test, label_train, label_test,data_valed, label_valed,path_test)


train (278, 3, 256, 256) test (124, 3, 256, 256) label [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,